# All Models, All Features, Target Encoding

In [1]:
%load_ext watermark
%watermark -p pandas,numpy,scikit-learn,matplotlib --conda 

pandas      : 1.4.4
numpy       : 1.23.3
scikit-learn: 1.1.2
matplotlib  : 3.5.3

conda environment: hotspot



In [1]:
import pandas as pd
import numpy as np

## Binarize

In [2]:
df_train = pd.read_csv('../datasets/TrainingDataset.csv')
df_test = pd.read_csv('../datasets/3class-TestDataset.csv')

# 1 & 2 -> 1
# 0 -> 0

binarized = df_train['3-class'].values.copy()
binarized[binarized == 2] = 1
df_train['2-class-merged-v1'] = binarized.astype(int)

binarized = df_test['3-class'].values.copy()
binarized[binarized == 2] = 1
df_test['2-class-merged-v1'] = binarized.astype(int)


y_train = df_train['2-class-merged-v1'].values
y_test = df_test['2-class-merged-v1'].values

## Use All Features

In [3]:
np.unique(df_train['residue'])

array(['C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q',
       'R', 'S', 'T', 'V', 'W', 'Y'], dtype=object)

In [4]:
# convert aa char to int
codes = ['A', 'R', 'N', 'D', 'C', 'E', 'Q', 'G', 'H', 
         'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']
code_to_int = {c:i for i,c in enumerate(codes)}     
df_train['residue'] = df_train['residue'].map(code_to_int)
df_test['residue'] = df_test['residue'].map(code_to_int)

In [5]:
np.unique(df_train['secondary structure'])

array(['-', 'H', 'S', 'T'], dtype=object)

In [6]:
# convert secondary structure char to int
codes = ['H', 'S', 'T', '-']
code_to_int = {c:i for i,c in enumerate(codes)}     
df_train['secondary structure'] = df_train['secondary structure'].map(code_to_int)
df_test['secondary structure'] = df_test['secondary structure'].map(code_to_int)

feature_list = ['avg bond number', 'Hbond', 'residue',
                'Hphob', 'consurf', "B' side chain", 'secondary structure', 'asa']

df_train = df_train[feature_list]
df_test = df_test[feature_list]

In [7]:
df_train['residue'] = df_train['residue'].astype('category')
df_test['residue'] = df_test['residue'].astype('category')

df_train['secondary structure'] = df_train['secondary structure'].astype('category')
df_test['secondary structure'] = df_test['secondary structure'].astype('category')

In [8]:
df_train.dtypes

avg bond number         float64
Hbond                     int64
residue                category
Hphob                     int64
consurf                   int64
B' side chain           float64
secondary structure    category
asa                     float64
dtype: object

In [9]:
X_train = df_train[feature_list].values
X_test =  df_test[feature_list].values

In [10]:
X_train.shape

(732, 8)

In [11]:
y_train.shape

(732,)

In [12]:
X_test.shape

(314, 8)

In [13]:
y_test.shape

(314,)

In [14]:
X_train[:, [2, 6]]

array([[13.,  3.],
       [ 1.,  1.],
       [14.,  3.],
       ...,
       [19.,  2.],
       [14.,  2.],
       [ 1.,  3.]])

## Target Encoding

In [15]:
from dirty_cat import TargetEncoder

enc = TargetEncoder(categories='auto', clf_type='binary-clf')
enc.fit(df_train[feature_list][['residue', 'secondary structure']],
        y_train)

TargetEncoder()

In [16]:
enc.transform(df_train[feature_list][['residue', 'secondary structure']])

array([[0.53006536, 0.44274376],
       [0.46017334, 0.42806268],
       [0.38758993, 0.44274376],
       ...,
       [0.45571537, 0.43303571],
       [0.38758993, 0.43303571],
       [0.46017334, 0.44274376]])

In [17]:
df_train_ohe = df_train.drop(columns=['residue', 'secondary structure'])
df_test_ohe = df_test.drop(columns=['residue', 'secondary structure'])

In [18]:
ohe_train = enc.transform(df_train[feature_list][['residue', 'secondary structure']])
ohe_test = enc.transform(df_test[feature_list][['residue', 'secondary structure']])

In [19]:
X_train_ohe = np.hstack((df_train_ohe.values, ohe_train))
X_test_ohe = np.hstack((df_test_ohe.values, ohe_test))

## Logistic Regression

### Baseline

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score



pipe = make_pipeline(StandardScaler(),
                     LogisticRegression(random_state=123))


score_names = ['precision', 'recall', 'f1']

for score_name in score_names:

    scores = cross_val_score(
        X=X_train_ohe,
        y=y_train,
        estimator=pipe,
        cv=10,
        scoring=score_name
    )
    
    print(f'{score_name}: {np.mean(scores):.2f} +/- {np.std(scores):.2f}')

precision: 0.60 +/- 0.06
recall: 0.56 +/- 0.11
f1: 0.58 +/- 0.08


### Tuned

In [21]:
import scipy.stats
import sklearn
from sklearn.model_selection import GridSearchCV


pipe = make_pipeline(StandardScaler(),
                     LogisticRegression(random_state=123, max_iter=1000))

params =  {
    'logisticregression__penalty':['l1', 'l2'],
    'logisticregression__C': [0.0001, 0.001, 0.01, 0.1, 0.0, 1.0, 10, 100, 1000],
}

gs = GridSearchCV(estimator=pipe, 
                  param_grid=params, 
                  scoring='f1', 
                  refit=True,
                  cv=10)

gs = gs.fit(X_train_ohe, y_train)
print('CV F1 score', gs.best_score_)
print('Best params', gs.best_params_)


score_names = ['precision', 'recall', 'f1']


for score_name in score_names:

    scores = cross_val_score(
        X=X_train_ohe,
        y=y_train,
        estimator=gs.best_estimator_,
        cv=10,
        n_jobs=-1,
        scoring=score_name
    )
    print(f'10-fold CV {score_name}: {np.mean(scores):.2f} +/- {np.std(scores):.2f}')

    
for score_name in score_names:
    
    scorer = sklearn.metrics.get_scorer(score_name)
    
    score = scorer(gs.best_estimator_, X_test_ohe, y_test)
    print(f'Test {score_name}: {score:.2f}')

CV F1 score 0.5766408706974199
Best params {'logisticregression__C': 1.0, 'logisticregression__penalty': 'l2'}


/Users/sebastian/miniforge3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
100 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/sebastian/miniforge3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/sebastian/miniforge3/lib/python3.9/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/sebastian/miniforge3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1461, in fit


10-fold CV precision: 0.60 +/- 0.06
10-fold CV recall: 0.56 +/- 0.11
10-fold CV f1: 0.58 +/- 0.08
Test precision: 0.63
Test recall: 0.59
Test f1: 0.61


## Random Forest OneHot

### Baseline

In [22]:
from sklearn.ensemble import RandomForestClassifier


score_names = ['precision', 'recall', 'f1']

for score_name in score_names:

    scores = cross_val_score(
        X=X_train_ohe,
        y=y_train,
        estimator=RandomForestClassifier(random_state=123, n_estimators=1000),
        cv=10,
        n_jobs=-1,
        scoring=score_name
    )
    
    print(f'{score_name}: {np.mean(scores):.2f} +/- {np.std(scores):.2f}')

precision: 0.59 +/- 0.08
recall: 0.53 +/- 0.09
f1: 0.56 +/- 0.08


### Tuned

In [23]:
pipe = make_pipeline(RandomForestClassifier(random_state=123))

params =  {
    'randomforestclassifier__max_depth': [3, 5, 10],
    'randomforestclassifier__min_samples_split': [2, 5, 10],
    'randomforestclassifier__n_estimators': [1000],
}

gs = GridSearchCV(estimator=pipe, 
                  param_grid=params, 
                  scoring='f1', 
                  refit=True,
                  cv=10)

gs = gs.fit(X_train_ohe, y_train)
print('CV F1 score', gs.best_score_)
print('Best params', gs.best_params_)


score_names = ['precision', 'recall', 'f1']


for score_name in score_names:

    scores = cross_val_score(
        X=X_train_ohe,
        y=y_train,
        estimator=gs.best_estimator_,
        cv=10,
        n_jobs=-1,
        scoring=score_name
    )
    print(f'10-fold CV {score_name}: {np.mean(scores):.2f} +/- {np.std(scores):.2f}')

    
for score_name in score_names:
    
    scorer = sklearn.metrics.get_scorer(score_name)
    
    score = scorer(gs.best_estimator_, X_test_ohe, y_test)
    print(f'Test {score_name}: {score:.2f}')

CV F1 score 0.5915128287758064
Best params {'randomforestclassifier__max_depth': 5, 'randomforestclassifier__min_samples_split': 5, 'randomforestclassifier__n_estimators': 1000}
10-fold CV precision: 0.62 +/- 0.07
10-fold CV recall: 0.58 +/- 0.13
10-fold CV f1: 0.59 +/- 0.09
Test precision: 0.64
Test recall: 0.58
Test f1: 0.61


## HistGradientBoosting

In [24]:
from sklearn.ensemble import HistGradientBoostingClassifier


pipe = make_pipeline(HistGradientBoostingClassifier(random_state=123))

params = {
    'histgradientboostingclassifier__learning_rate': (0.01, 0.1, 1, 10),
    'histgradientboostingclassifier__max_leaf_nodes': (3, 10, 30)
}

gs = GridSearchCV(estimator=pipe, 
                  param_grid=params, 
                  scoring='f1', 
                  refit=True,
                  cv=10)

gs = gs.fit(X_train_ohe, y_train)
print('CV F1 score', gs.best_score_)
print('Best params', gs.best_params_)


score_names = ['precision', 'recall', 'f1']


for score_name in score_names:

    scores = cross_val_score(
        X=X_train_ohe,
        y=y_train,
        estimator=gs.best_estimator_,
        cv=10,
        n_jobs=-1,
        scoring=score_name
    )
    print(f'10-fold CV {score_name}: {np.mean(scores):.2f} +/- {np.std(scores):.2f}')

    
for score_name in score_names:
    
    scorer = sklearn.metrics.get_scorer(score_name)
    
    score = scorer(gs.best_estimator_, X_test_ohe, y_test)
    print(f'Test {score_name}: {score:.2f}')

CV F1 score 0.5893115809151049
Best params {'histgradientboostingclassifier__learning_rate': 0.1, 'histgradientboostingclassifier__max_leaf_nodes': 3}
10-fold CV precision: 0.60 +/- 0.10
10-fold CV recall: 0.59 +/- 0.16
10-fold CV f1: 0.59 +/- 0.12
Test precision: 0.62
Test recall: 0.61
Test f1: 0.62


## Support vector machine

In [25]:
from sklearn.svm import SVC


pipe = make_pipeline(StandardScaler(), SVC(random_state=123))

param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

params = [{'svc__C': param_range, 
               'svc__kernel': ['linear']},
              {'svc__C': param_range, 
               'svc__gamma': param_range, 
               'svc__kernel': ['rbf']}]

gs = GridSearchCV(estimator=pipe, 
                  param_grid=params, 
                  scoring='f1', 
                  refit=True,
                  cv=10)

gs = gs.fit(X_train_ohe, y_train)
print('CV F1 score', gs.best_score_)
print('Best params', gs.best_params_)


score_names = ['precision', 'recall', 'f1']


for score_name in score_names:

    scores = cross_val_score(
        X=X_train_ohe,
        y=y_train,
        estimator=gs.best_estimator_,
        cv=10,
        n_jobs=-1,
        scoring=score_name
    )
    print(f'10-fold CV {score_name}: {np.mean(scores):.2f} +/- {np.std(scores):.2f}')

    
for score_name in score_names:
    
    scorer = sklearn.metrics.get_scorer(score_name)
    
    score = scorer(gs.best_estimator_, X_test_ohe, y_test)
    print(f'Test {score_name}: {score:.2f}')

CV F1 score 0.6040465114472755
Best params {'svc__C': 1000.0, 'svc__gamma': 0.01, 'svc__kernel': 'rbf'}
10-fold CV precision: 0.63 +/- 0.08
10-fold CV recall: 0.59 +/- 0.10
10-fold CV f1: 0.60 +/- 0.08
Test precision: 0.62
Test recall: 0.60
Test f1: 0.61


## Multilayer Perceptron

In [27]:
from sklearn.neural_network import MLPClassifier


pipe = make_pipeline(StandardScaler(),
                     MLPClassifier(max_iter=10000, random_state=123))


params = {
    'mlpclassifier__hidden_layer_sizes': [(30, 20, 10), 
                                          (40, 20), 
                                          (20, 10), 
                                          (20,),
                                          (10,)],
    'mlpclassifier__activation': ['tanh', 'relu'],
    'mlpclassifier__solver': ['sgd', 'adam'],
    'mlpclassifier__alpha': [0.0001, 0.001, 0.05],
    'mlpclassifier__learning_rate': ['constant', 'adaptive'],
}

gs = GridSearchCV(estimator=pipe, 
                  param_grid=params, 
                  scoring='f1', 
                  refit=True,
                  n_jobs=-1,
                  cv=10)

gs = gs.fit(X_train_ohe, y_train)
print('CV F1 score', gs.best_score_)
print('Best params', gs.best_params_)


score_names = ['precision', 'recall', 'f1']


for score_name in score_names:

    scores = cross_val_score(
        X=X_train_ohe,
        y=y_train,
        estimator=gs.best_estimator_,
        cv=10,
        n_jobs=-1,
        scoring=score_name
    )
    print(f'10-fold CV {score_name}: {np.mean(scores):.2f} +/- {np.std(scores):.2f}')

CV F1 score 0.618805623226676
Best params {'mlpclassifier__activation': 'relu', 'mlpclassifier__alpha': 0.0001, 'mlpclassifier__hidden_layer_sizes': (40, 20), 'mlpclassifier__learning_rate': 'constant', 'mlpclassifier__solver': 'sgd'}
10-fold CV precision: 0.62 +/- 0.05
10-fold CV recall: 0.63 +/- 0.10
10-fold CV f1: 0.62 +/- 0.07


NameError: name 'X_test_oh' is not defined

In [28]:
for score_name in score_names:
    
    scorer = sklearn.metrics.get_scorer(score_name)
    
    score = scorer(gs.best_estimator_, X_test_ohe, y_test)
    print(f'Test {score_name}: {score:.2f}')

Test precision: 0.63
Test recall: 0.66
Test f1: 0.64
